# Launch Training Jobs on Azure Cluster

In [1]:
from azureml.core import Workspace
from azureml.exceptions import WorkspaceException
from azureml.exceptions import ComputeTargetException
from azureml.core.compute import ComputeTarget
from azureml.core.compute import AmlCompute
from azureml.core import Experiment
from azureml.core import Datastore
from azureml.train.estimator import Estimator
from azureml.tensorboard import Tensorboard
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive import GridParameterSampling
from azureml.train.hyperdrive import PrimaryMetricGoal
from azureml.train.hyperdrive import MedianStoppingPolicy
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.parameter_expressions import uniform

from pathlib import Path

## Options

In [2]:
LIST_VMS = False
USE_GPU = True

## Create Workspace

In [3]:
try:
    ws = Workspace.create(name='replearn',
                          location='eastus',
                          resource_group='rg1',
                          subscription_id='5fb52191-233d-4b0f-9713-de0e41784e6e')
    ws.write_config()
except WorkspaceException:
    print('Found existing Workspace, using it.')
    ws = Workspace.from_config(Path.cwd() / '.azureml' / 'config.json')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found existing Workspace, using it.


In [4]:
if LIST_VMS:
    print(AmlCompute.supported_vmsizes(workspace=ws))

## Create Compute Target (Cluster)
A persistent Azure Machine Learning Compute can be reused across jobs. The compute can be shared with other users in the workspace and is kept between jobs.  
https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets  

In [5]:
# Set cluster name
cluster_name = "NC6-cluster4"
vm_priority = 'dedicated'  # dedicated or lowpriority
max_nodes = 4

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, using it.')
except ComputeTargetException:
    if USE_GPU:
        vm_size = 'Standard_NC6s_v3'
    else:
        vm_size = 'Standard_DS4_v2'
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=max_nodes,
                                                           vm_priority=vm_priority,
                                                           idle_seconds_before_scaledown=3600)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Found existing cluster, using it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


## Register Datastore  
https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data

In [6]:
datastores = ws.datastores
if 'training_data' not in datastores.keys():
    acct_key = 'fzVL5O3ybeVQ/eBeMzp5YqnmDUkFwhVJTWPaNezIuxAZWoduY79W7o3l3Zop3FN22txHCXl3UBkdaaM/9C+12Q=='
    Datastore.register_azure_blob_container(workspace=ws, 
                                            datastore_name='training_data', 
                                            container_name='asgdata',
                                            account_name='asgdata', 
                                            account_key=acct_key,
                                            create_if_not_exists=False)
else:
    print('Found existing training_data Datastore, using it.')
ds = ws.datastores['training_data']

Found existing training_data Datastore, using it.


## Create an Experiment and Run  
https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-ml-models  

In [7]:
# `train_dir` and `val_dir` are path on container to training and val `*.tfr` files
script_params = {
    '--azure_ml': "",
    '--train_dir': ds.path('Data/LibriSpeech/tfrecords/train-clean-100').as_download(),
    '--val_dir': ds.path('Data/LibriSpeech/tfrecords/dev-clean').as_download()
}

tf_est = Estimator(source_directory=Path.cwd() / '..' / 'replearn',
                   script_params=script_params,
                   compute_target=cluster,
                   use_gpu=USE_GPU,
                   entry_script='train.py',
                   pip_packages=['tensorflow-gpu==2.0.0'])

In [8]:
experiment_name = 'replearn'
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
run = exp.submit(tf_est)
print(run.get_portal_url())
run.wait_for_completion(show_output=True)  # view stream of stdout

https://mlworkspace.azure.ai/portal/subscriptions/5fb52191-233d-4b0f-9713-de0e41784e6e/resourceGroups/rg1/providers/Microsoft.MachineLearningServices/workspaces/replearn/experiments/replearn/runs/replearn_1573674050_838294b2
RunId: replearn_1573674050_838294b2
Web View: https://mlworkspace.azure.ai/portal/subscriptions/5fb52191-233d-4b0f-9713-de0e41784e6e/resourceGroups/rg1/providers/Microsoft.MachineLearningServices/workspaces/replearn/experiments/replearn/runs/replearn_1573674050_838294b2

Streaming azureml-logs/20_image_build_log.txt

2019/11/13 19:40:57 Downloading source code...
2019/11/13 19:40:59 Finished downloading source code
2019/11/13 19:40:59 Creating Docker network: acb_default_network, driver: 'bridge'
2019/11/13 19:40:59 Successfully set up Docker network: acb_default_network
2019/11/13 19:40:59 Setting up Docker configuration...
2019/11/13 19:41:00 Successfully set up Docker configuration
2019/11/13 19:41:00 Logging in to registry: replearn798858f2.azurecr.io
2019/11/1

## Launch TensorBoard Server  
https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-monitor-tensorboard

In [ ]:
tb = Tensorboard([run])
tb.start() # click on link and tensboard up-and-running with run training data

In [ ]:
# close server when done
tb.stop()

## Hyperparameter Tuning: Grid Search  
https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters

In [13]:
param_sampling = GridParameterSampling( {
        "batch_sz": choice(32, 64, 128),
        "optimizer": choice('adam', 'sgd_mom'),
        "lr": choice('0.001', '0.01', '0.1')})

hyperdrive_run_config = HyperDriveConfig(estimator=tf_est,
                          hyperparameter_sampling=param_sampling, 
                          primary_metric_name="val_loss",
                          policy=MedianStoppingPolicy(evaluation_interval=5, delay_evaluation=5),
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                          max_total_runs=100,
                          max_concurrent_runs=max_nodes)

In [14]:
experiment = Experiment(ws, 'hyperparam')
hyperdrive_run = experiment.submit(hyperdrive_run_config)
print(hyperdrive_run.get_portal_url())

https://mlworkspace.azure.ai/portal/subscriptions/5fb52191-233d-4b0f-9713-de0e41784e6e/resourceGroups/rg1/providers/Microsoft.MachineLearningServices/workspaces/replearn/experiments/hyperparam/runs/hyperparam_1570471202673140
